In [1]:
import pandas as pd
import dbconnect  # dbconnect.py 모듈 활용
import os

def create_station_mapping():
    print("🚀 [Master Data] 호선 매핑 테이블 생성 시작...")
    
    # 1. DB 연결 (seoul_urban_lab)
    # dbconnect 모듈을 통해 안전하게 접속
    conn = dbconnect.MydbConnect('seoul_urban_lab')
    
    try:
        # 2. 쿼리 작성
        # 중복을 제거(DISTINCT)하여 역코드-역명-호선명 조합만 가져옵니다.
        # stnNm은 검증용으로 같이 가져옵니다.
        sql = """
            SELECT DISTINCT stnCd, stnNm, lineNm 
            FROM subway_traffic_log
            WHERE lineNm IS NOT NULL AND lineNm != ''
            ORDER BY stnCd
        """
        
        # 3. 데이터 조회 및 DataFrame 변환
        # pandas의 read_sql을 사용하면 DB 데이터를 바로 DF로 만들 수 있습니다.
        df_mapping = pd.read_sql(sql, conn)
        
        if df_mapping.empty:
            print("❌ DB에 데이터가 없습니다. 먼저 API 수집(run_etl.py)을 완료해주세요.")
            return

        # 4. 데이터 정제 (혹시 모를 중복 제거)
        # stnCd가 같은데 호선이 여러 개인 경우(환승역 등)를 대비해
        # 가장 최근에 수집된 정보를 신뢰하거나, 중복을 그대로 둡니다.
        # 여기서는 매핑의 정확도를 위해 중복을 제거하되, 첫 번째 발견된 것을 유지합니다.
        df_mapping = df_mapping.drop_duplicates(subset=['stnCd'], keep='first')
        
        # 5. CSV 파일로 저장
        file_name = 'station_master_mapping.csv'
        df_mapping.to_csv(file_name, index=False, encoding='utf-8-sig')
        
        print(f"\n✅ 매핑 파일 생성 완료: {file_name}")
        print(f"   - 총 매핑 가능 역 개수: {len(df_mapping)}")
        print("-" * 50)
        
        # 미리보기
        pd.set_option('display.max_columns', None)
        print(df_mapping.head(10))
        print("-" * 50)
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        
    finally:
        conn.close()

if __name__ == "__main__":
    create_station_mapping()

🚀 [Master Data] 호선 매핑 테이블 생성 시작...
데이터베이서 연결 시작!
MySQL DB 연결 성공

✅ 매핑 파일 생성 완료: station_master_mapping.csv
   - 총 매핑 가능 역 개수: 276
--------------------------------------------------
  stnCd         stnNm lineNm
0  0150           서울역    1호선
1  0151            시청    1호선
2  0152            종각    1호선
3  0153          종로3가    1호선
4  0154          종로5가    1호선
5  0155           동대문    1호선
6  0156           신설동    1호선
7  0157           제기동    1호선
8  0158  청량리(서울시립대입구)    1호선
9  0159           동묘앞    1호선
--------------------------------------------------


C:\Users\Admin\AppData\Local\Temp\ipykernel_15348\1339011976.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_mapping = pd.read_sql(sql, conn)


In [2]:
import os

def generate_project_report():
    filename = "Seoul_Subway_Project_Report.md"
    
    content = """# 🚇 서울시 지하철 이용 패턴 분석 프로젝트 (Technical Report)

**작성일:** 2026년 2월 2일  
**작성자:** Urban Data Scientist (Full Stack Developer)  
**프로젝트 목표:** 서울 열린데이터 광장 API 및 과거 CSV 데이터를 활용하여 도시의 시공간적 사용자 패턴(Spatio-Temporal User Pattern) 분석

---

## 1. 시스템 아키텍처 (System Architecture)

본 프로젝트는 **Hybrid Data Pipeline** 방식을 채택하여, 최신 데이터(API)와 과거 데이터(CSV)를 통합 관리합니다.

### 1.1 데이터 소스 (Data Source)
1.  **실시간/최신 데이터:** 서울 열린데이터 광장 `getStnPsgr` API (최근 7일)
2.  **과거 이력 데이터:** 서울교통공사 연도별 노인 승하차 인원 CSV (2022~2024년)

### 1.2 기술 스택 (Tech Stack)
* **Language:** Python 3.9+
* **Database:** MySQL (MariaDB 호환)
* **ETL Libraries:** `pandas`, `requests`, `pymysql`, `dbconnect` (Custom Module)
* **Environment:** `.env`를 이용한 보안 키 관리

---

## 2. 데이터베이스 스키마 설계 (Database Schema)

데이터의 무결성(Integrity)과 중복 방지(Idempotency)를 위해 복합 유니크 키(Unique Key)를 적용한 스키마를 설계하였습니다.

### 2.1 통합 로그 테이블 (`subway_traffic_log`)
API를 통해 수집된 최신 데이터를 저장하는 메인 테이블입니다.

| 컬럼명 | 데이터 타입 | 설명 | 비고 |
| :--- | :--- | :--- | :--- |
| `id` | BIGINT (PK) | 고유 식별자 | Auto Increment |
| `UserGroup` | VARCHAR | 사용자 그룹 | 일반, 청소년, 노인/약자 등 (파생변수) |
| `pasngDe` | VARCHAR(8) | 통행일자 | YYYYMMDD |
| `pasngHr` | INT | 통행시간 | 0 ~ 23 |
| `lineNm` | VARCHAR | 호선명 | |
| `stnCd` | VARCHAR | 역코드 | |
| `stnNm` | VARCHAR | 역명 | |
| `trnscdUserSeCd` | VARCHAR | 사용자 구분 코드 | API 원본 코드 유지 |
| `rideNope` | INT | 승차 인원 | |
| `gffNope` | INT | 하차 인원 | |
| **Unique Key** | (Composite) | `pasngDe`, `pasngHr`, `stnCd`, `trnscdUserSeCd` | 중복 적재 방지 (Upsert 기준) |

### 2.2 노인 데이터 전용 테이블 (`subway_traffic_log_senior_22-24`)
2022~2024년 CSV 파일(Wide Format)을 변환하여 저장하는 히스토리 테이블입니다. 스키마 구조는 위와 동일합니다.

---

## 3. ETL 파이프라인 구축 내용 (Data Pipeline)

### 3.1 API 수집 파이프라인 (The Daily Crawler)
* **Batch Size:** 1,000건 (안정성 테스트 완료, Timeout 30초 설정)
* **User Categorization Logic:**
    * `01`: 일반
    * `02`, `03`, `04`: 어린이/학생/청소년
    * `06`, `100`: 노인/약자
    * 그 외: 기타
* **Feature:** `dbconnect` 모듈을 통한 안전한 DB 연결, 날짜별 자동 순회 수집.

### 3.2 CSV 배치 파이프라인 (Historical Data Loader)
CSV 파일의 구조적 한계(Wide Format, 호선 정보 누락)를 극복하기 위한 전처리 로직을 적용했습니다.

1.  **Master Data 생성:** API 테이블(`subway_traffic_log`)에서 `stnCd`와 `lineNm` 매핑 정보를 추출하여 `station_master_mapping.csv` 생성.
2.  **Missing Data Filling:** CSV 파일 로드 시 매핑 테이블과 Left Join하여 누락된 `lineNm` 보완.
3.  **Reshaping:** `Melt` 함수를 사용하여 Wide Format(시간대가 컬럼으로 나열됨)을 Long Format(시간대가 행으로 내려옴)으로 변환.
4.  **Cleaning:** '06시간대이전' → 5, '24시간대이후' → 0 으로 정수화 처리.

---

## 4. 데이터 검증 결과 (Verification)

### 4.1 API 수집 검증
* **대상:** 2026년 1월 26일 ~ 2월 1일 데이터
* **결과:** 1,000건 단위 페이징(Pagination) 정상 작동, 전량 DB 적재 완료.

### 4.2 CSV 변환 검증
* **대상:** 2022, 2023, 2024년 노인 승하차 데이터
* **결과:** 인코딩(cp949/utf-8) 자동 처리, 호선 정보 매핑 완료.
* **샘플 확인:** 2022년 2월 23일 15시 '서울역' 데이터 조회 시 호선별 분리되어 정상 출력됨을 확인.

---

## 5. 향후 계획 (Next Steps)
1.  **자동화:** `run_etl.py`를 Airflow 또는 Crontab에 등록하여 매일 새벽 자동 수집.
2.  **시각화:** Streamlit을 연동하여 '도시의 리듬(시간대별 패턴)', '실버 존(노인 밀집 역)' 대시보드 구축.
3.  **고도화:** 날씨 데이터(기상청 API)와 조인하여 기상 조건에 따른 지하철 이용 패턴 변화 분석.

"""
    
    # UTF-8 인코딩으로 파일 저장
    with open(filename, "w", encoding="utf-8") as f:
        f.write(content)
    
    print(f"✅ 기획서 파일 생성 완료: {os.path.abspath(filename)}")
    print("이 파일을 열어서 내용을 확인하거나 공유하시면 됩니다.")

if __name__ == "__main__":
    generate_project_report()

✅ 기획서 파일 생성 완료: c:\Users\Admin\Desktop\hdc2\0202 프로젝트 구축\Seoul_Subway_Project_Report.md
이 파일을 열어서 내용을 확인하거나 공유하시면 됩니다.
